In [1]:
import numpy as np
import pandas as pd
import bcolz
import time
import logging
import datetime

import sys
sys.path.append('..')

from bcolzutils import *
from util import *

import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler
from keras import optimizers
from keras.regularizers import l2 

from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input


import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)




Using TensorFlow backend.
/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
arch = "vgg19"
basedir="/home/tutysara/src/myprojects/dog-project/dogImages"

bnf_valid_name = basedir + f'/bottleneck_features_{arch}_valid'
bnf_test_name = basedir + f'/bottleneck_features_{arch}_test' 
bnf_train_name = basedir + f'/bottleneck_features_{arch}_train'

percent = 0.005
percent = 1
epochs=15
num_classes = 133
batch_size = 64
lr=1e-3
momentum=0.9
weight_decay = 1e-5
test_prefix=""

def lr_schedule(epoch):
    """ divides the lr by 10 every 5 epochs"""
    n = epoch // 5
    return lr * (0.1 ** n)

if percent < 1:
    test_prefix = "_test"
    
test_result = f'bottleneck_features_{arch}_result{test_prefix}.npz'
model_path = f'../saved_models/weights.best.topmodel.{arch}{test_prefix}.hdf5'
loss_history_csv_name = f'train_top_model_{arch}_loss_history{test_prefix}.csv'

d = datetime.datetime.today()
log_filename=f"train_topmodel_bottleneck_{arch}_{d.year}-{d.month}-{d.day}-{d.hour}.{d.minute}.{d.second}{test_prefix}.log"

logging.basicConfig(level='DEBUG',
                    handlers=[logging.FileHandler(log_filename),
                              logging.StreamHandler()])

log = logging.getLogger(__name__)
log.debug("fit and save top mode using bottleneck features")
log.debug("using top_model_weight_path" + model_path)
log.debug("using test_result" + test_result)
log.debug("using loss_history_csv_name" + loss_history_csv_name)

DEBUG:__main__:fit and save top mode using bottleneck features
DEBUG:__main__:using top_model_weight_path../saved_models/weights.best.topmodel.vgg19.hdf5
DEBUG:__main__:using test_resultbottleneck_features_vgg19_result.npz
DEBUG:__main__:using loss_history_csv_nametrain_top_model_vgg19_loss_history.csv


In [3]:
# define function to load train, test, and validation datasets
from sklearn.datasets import load_files 
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset(basedir + '/../' +'dogImages/train')
valid_files, valid_targets = load_dataset(basedir + '/../' +'dogImages/valid')
test_files, test_targets = load_dataset(basedir + '/../' +'dogImages/test')

In [4]:
# convert and load images
from keras.preprocessing import image                  
from tqdm import tqdm
# load and preprocess data
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True  

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

               

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')
valid_tensors = paths_to_tensor(valid_files).astype('float32')
test_tensors = paths_to_tensor(test_files).astype('float32')

100%|██████████| 836/836 [00:07<00:00, 107.76it/s]


In [ ]:
print(train_tensors.shape)
print(valid_tensors.shape) 
print(test_tensors.shape) 

In [ ]:
from keras.applications.vgg19 import VGG19, preprocess_input

bnf_train_name = 'bottleneck_features_vgg19_train.npy'
bnf_test_name = 'bottleneck_features_vgg19_test.npy' 
bnf_valid_name = 'bottleneck_features_vgg19_valid.npy'

vgg19_feature_ext = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

bottleneck_features_train = vgg19_feature_ext.predict(preprocess_input(train_tensors))
np.save(open(bnf_train_name, 'wb'),bottleneck_features_train)

bottleneck_features_validation = vgg19_feature_ext.predict(preprocess_input(valid_tensors))
np.save(open(bnf_test_name, 'wb'),bottleneck_features_validation)

bottleneck_features_test = vgg19_feature_ext.predict(preprocess_input(test_tensors))
np.save(open(bnf_valid_name, 'wb'),bottleneck_features_test)

In [ ]:
# load convered data back
bnf_train_data = np.load(open(bnf_train_name, 'rb'))
bnf_validation_data = np.load(open(bnf_test_name, 'rb'))
bnf_test_data = np.load(open(bnf_valid_name, 'rb'))
print(bnf_train_data.shape)
print(bnf_validation_data.shape)
print(bnf_test_data.shape)
## go to train model

In [ ]:
## top model
classes = num_classes

  
top_model = Sequential()
top_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 512)))
#top_model.add(Flatten(input_shape=(7, 7, 512)))
top_model.add(Dense(4096, activation='relu', name='fc1'))
top_model.add(Dropout(0.5, name='fc1-dropout'))
top_model.add(Dense(4096, activation='relu', name='fc2'))
top_model.add(Dropout(0.5, name='fc2-dropout'))
top_model.add(Dense(classes, activation='softmax', name='predictions'))

#top_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

top_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])
           
"""             
top_model = Sequential()
top_model.add(Flatten(input_shape=(7, 7, 512)))
#top_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 512)))
top_model.add(Dropout(0.2))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.2))
top_model.add(Dense(133, activation='softmax'))

top_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])
""" 
top_model.summary()


In [ ]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

top_model.fit(bnf_train_data, train_targets,
          epochs=epochs,
          validation_data=(bnf_validation_data, valid_targets),
          callbacks=[early_stopping, lrscheduler])

In [ ]:
bottleneck_features = np.load(basedir + '/../' + 'bottleneck_features/DogVGG19Data.npz')
train_VGG19 = bottleneck_features['train']
valid_VGG19 = bottleneck_features['valid']
test_VGG19 = bottleneck_features['test']

print(train_VGG19.shape, train_targets.shape)
print(valid_VGG19.shape, valid_targets.shape)
print(test_VGG19.shape, test_targets.shape)

In [ ]:
top_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])

top_model.fit(train_VGG19, train_targets,
          epochs=epochs,
          validation_data=(valid_VGG19, valid_targets),
          callbacks=[early_stopping, lrscheduler])

In [ ]:
#basedir="/media/hdd/datastore/t4sa"
batch_size = 128
bnf_valid_name = basedir + f'/bottleneck_features_{arch}_valid'
bnf_test_name = basedir + f'/bottleneck_features_{arch}_test' 
bnf_train_name = basedir + f'/bottleneck_features_{arch}_train'

## Read it back from disk and check size
bnf_valid_data = bcolz.carray(rootdir=f'{bnf_valid_name}_data.bclz', mode='r')
bnf_test_data = bcolz.carray(rootdir=f'{bnf_test_name}_data.bclz', mode='r')
bnf_train_data = bcolz.carray(rootdir=f'{bnf_train_name}_data.bclz', mode='r')

bnf_valid_labels = bcolz.carray(rootdir=f'{bnf_valid_name}_labels.bclz', mode='r')
bnf_test_labels = bcolz.carray(rootdir=f'{bnf_test_name}_labels.bclz', mode='r')
bnf_train_labels = bcolz.carray(rootdir=f'{bnf_train_name}_labels.bclz', mode='r')

log.debug(bnf_valid_data.shape)
log.debug(bnf_test_data.shape)
log.debug(bnf_train_data.shape)

log.debug(bnf_valid_labels.shape)
log.debug(bnf_test_labels.shape)
log.debug(bnf_train_labels.shape)

bnf_valid_data_size = int(bnf_valid_data.shape[0]*percent)
bnf_test_data_size = int(bnf_test_data.shape[0]*percent)
bnf_train_data_size = int(bnf_train_data.shape[0]*percent)

if percent < 1:
    bnf_valid_data = bnf_valid_data[:bnf_valid_data_size]
    bnf_valid_labels = bnf_valid_labels[:bnf_valid_data_size]
    
    bnf_test_data = bnf_test_data[:bnf_test_data_size]
    bnf_test_labels = bnf_test_labels[:bnf_test_data_size]
    
    bnf_train_data = bnf_train_data[:bnf_train_data_size]
    bnf_train_labels = bnf_train_labels[:bnf_train_data_size]

log.debug("loading percentage of original data from disk")
log.debug(bnf_valid_data.shape)
log.debug(bnf_test_data.shape)
log.debug(bnf_train_data.shape)

log.debug(bnf_valid_labels.shape)
log.debug(bnf_test_labels.shape)
log.debug(bnf_train_labels.shape)

bnf_train_gen =bcolz_data_generator(bnf_train_data, bnf_train_labels, batch_size=batch_size)
bnf_valid_gen =bcolz_data_generator(bnf_valid_data, bnf_valid_labels, batch_size=batch_size)
bnf_test_gen =bcolz_data_generator(bnf_test_data, bnf_test_labels, batch_size=batch_size)

In [ ]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

top_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])

top_model.fit_generator(bnf_train_gen,
          steps_per_epoch= (1 + int(bnf_train_data_size // batch_size)),
          epochs=epochs,
          validation_data=bnf_valid_gen,
          validation_steps= (1 + int(bnf_valid_data_size // batch_size)),
          callbacks=[early_stopping, lrscheduler])


In [7]:
# Generate a model with all layers (with top)
vgg19 = VGG19(weights='imagenet', include_top=True)

#Add a layer where input is the output of the  second last layer 
x = Dense(num_classes, activation='softmax', name='my_predictions')(vgg19.layers[-2].output)

for layer in vgg19.layers:
    layer.trainable = False
    
#Then create the corresponding model 
my_model = Model(input=vgg19.input, output=x)
my_model.layers[-1].trainable = True
my_model.layers[-2].trainable = True
my_model.layers[-3].trainable = True
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("my...)`
  # This is added back by InteractiveShellApp.init_path()


In [8]:
for layer in my_model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_conv4 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_conv4 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_conv4 False
block5_pool False
flatten False
fc1 True
fc2 True
my_predictions True


In [ ]:
valid_data = vgg19_preprocess_input(valid_tensors, mode='caffe')
test_data = vgg19_preprocess_input(test_tensors, mode='caffe')
train_data = vgg19_preprocess_input(train_tensors, mode='caffe')

In [ ]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

my_model.fit(train_data, train_targets,
          epochs=epochs,
          validation_data=(valid_data, valid_targets),
          callbacks=[early_stopping, lrscheduler])

In [ ]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])

my_model.fit(train_data, train_targets,
          epochs=epochs,
          validation_data=(valid_data, valid_targets),
          callbacks=[early_stopping, lrscheduler])

In [ ]:
valid_data = vgg19_preprocess_input(valid_tensors, mode='tf')
test_data = vgg19_preprocess_input(test_tensors, mode='tf')
train_data = vgg19_preprocess_input(train_tensors, mode='tf')

checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])

my_model.fit(train_data, train_targets,
          epochs=epochs,
          validation_data=(valid_data, valid_targets),
          callbacks=[early_stopping, lrscheduler])

In [5]:
valid_data = vgg19_preprocess_input(valid_tensors, mode='caffe')
test_data = vgg19_preprocess_input(test_tensors, mode='caffe')
train_data = vgg19_preprocess_input(train_tensors, mode='caffe')

In [9]:
train_gen =bcolz_data_generator(train_data, train_targets, batch_size=batch_size)
valid_gen =bcolz_data_generator(valid_data, valid_targets, batch_size=batch_size)
test_gen =bcolz_data_generator(test_data, test_targets, batch_size=batch_size)

checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])

my_model.fit_generator(train_gen,
          steps_per_epoch= (1 + int(train_data.shape[0] // batch_size)),
          epochs=epochs,
          validation_data=valid_gen,
          validation_steps= (1 + int(valid_data.shape[0] // batch_size)),
          callbacks=[early_stopping, lrscheduler])

Epoch 1/15
105/105 [==============================] - 61s 579ms/step - loss: 1.5842 - acc: 0.6283 - val_loss: 0.8993 - val_acc: 0.7317
Epoch 2/15
105/105 [==============================] - 56s 532ms/step - loss: 0.2134 - acc: 0.9432 - val_loss: 0.8378 - val_acc: 0.7760
Epoch 3/15
105/105 [==============================] - 56s 534ms/step - loss: 0.0632 - acc: 0.9856 - val_loss: 0.7427 - val_acc: 0.7832
Epoch 4/15
105/105 [==============================] - 56s 534ms/step - loss: 0.0272 - acc: 0.9958 - val_loss: 0.7280 - val_acc: 0.7916
Epoch 5/15
105/105 [==============================] - 56s 533ms/step - loss: 0.0168 - acc: 0.9975 - val_loss: 0.7168 - val_acc: 0.7928
Epoch 6/15
105/105 [==============================] - 56s 534ms/step - loss: 0.0114 - acc: 0.9981 - val_loss: 0.6984 - val_acc: 0.7940
Epoch 7/15
105/105 [==============================] - 56s 534ms/step - loss: 0.0062 - acc: 0.9991 - val_loss: 0.6966 - val_acc: 0.7928
Epoch 8/15
105/105 [==============================] - 5